# Andre Marroquin 22266
# Sergio orellana 221122
# Carlos Valladares
# Rodrigo Mansilla

#  Preparación del entorno y carga de datos

In [18]:
import pandas as pd
import numpy as np
import unicodedata

# Cargar el archivo Excel
df = pd.read_excel('Establecimientos.xlsx')

# Vista rápida
print(f"Dimensiones: {df.shape[0]} filas, {df.shape[1]} columnas")
df.head(5)

Dimensiones: 6590 filas, 18 columnas


,CODIGO,Unnamed: 1,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL
0,16-01-0138-46,NaN,16-031,ALTA VERAPAZ,COBAN,COLEGIO COBAN,KM.2 SALIDA A SAN JUAN CHAMELCO ZONA 8,77945104,PATRICIO NAJARRO ASENCIO,GUSTAVO ADOLFO SIERRA POP,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
1,16-01-0139-46,NaN,16-031,ALTA VERAPAZ,COBAN,COLEGIO PARTICULAR MIXTO VERAPAZ,KM 209.5 ENTRADA A LA CIUDAD,77367402,PATRICIO NAJARRO ASENCIO,GILMA DOLORES GUAY PAZ DE LEAL,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
2,16-01-0140-46,NaN,16-031,ALTA VERAPAZ,COBAN,"COLEGIO ""LA INMACULADA""",7A. AVENIDA 11-109 ZONA 6,78232301,PATRICIO NAJARRO ASENCIO,VIRGINIA SOLANO SERRANO,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
3,16-01-0141-46,NaN,16-005,ALTA VERAPAZ,COBAN,ESCUELA NACIONAL DE CIENCIAS COMERCIALES,2A CALLE 11-10 ZONA 2,79514215,NORA LILIANA FIGUEROA HERNÁNDEZ,HÉCTOR ROLANDO CHUN POOU,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
4,16-01-0142-46,NaN,16-005,ALTA VERAPAZ,COBAN,INSTITUTO NORMAL MIXTO DEL NORTE 'EMILIO ROSAL...,3A AVE 6-23 ZONA 11,79521468,NORA LILIANA FIGUEROA HERNÁNDEZ,VICTOR HUGO DOMÍNGUEZ REYES,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,BILINGUE,VESPERTINA,DIARIO(REGULAR),ALTA VERAPAZ


# Descripción inicial del conjunto de datos

Filas iniciales: 6 590

Variables (columnas): 18

Nombres de columnas: ['CODIGO', 'Unnamed: 1', 'DISTRITO', 'DEPARTAMENTO', 'MUNICIPIO',
 'ESTABLECIMIENTO', 'DIRECCION', 'TELEFONO', 'SUPERVISOR', 'DIRECTOR',
 'NIVEL', 'SECTOR', 'AREA', 'STATUS', 'MODALIDAD', 'JORNADA', 'PLAN',
 'DEPARTAMENTAL']


In [19]:
# Conteo de valores faltantes por columna
missing = df.isna().sum().sort_values(ascending=False)
print("Valores faltantes por columna:\n", missing, "\n")



Valores faltantes por columna:
 Unnamed: 1         6590
TELEFONO             45
DIRECTOR             25
DIRECCION             2
SECTOR                0
PLAN                  0
JORNADA               0
MODALIDAD             0
STATUS                0
AREA                  0
CODIGO                0
NIVEL                 0
SUPERVISOR            0
ESTABLECIMIENTO       0
MUNICIPIO             0
DEPARTAMENTO          0
DISTRITO              0
DEPARTAMENTAL         0
dtype: int64 



In [20]:
# Verificación puntual
print(f"Unnamed: 1 faltantes: {missing['Unnamed: 1']}")
print(f"DIRECCION   faltantes: {missing['DIRECCION']}")
print(f"TELEFONO    faltantes: {missing['TELEFONO']}")
print(f"DIRECTOR    faltantes: {missing['DIRECTOR']}\n")

Unnamed: 1 faltantes: 6590
DIRECCION   faltantes: 2
TELEFONO    faltantes: 45
DIRECTOR    faltantes: 25



In [21]:
# Columnas constantes 
# Vacías todas las filas NaN
# Con un único valor
n_uniques = df.nunique(dropna=False)
const_empty = n_uniques[n_uniques == 0].index.tolist()
const_one   = n_uniques[n_uniques == 1].index.tolist()

print("Columnas constantes:")
print("Columnas completamente vacías:", const_empty)
print("Columnas con un único valor:", const_one, "\n")

Columnas constantes:
Columnas completamente vacías: []
Columnas con un único valor: ['Unnamed: 1', 'NIVEL', 'STATUS'] 



In [22]:
# Variables con más necesidad de limpieza
top_missing = missing.head(5).index.tolist()
text_cols = df.select_dtypes(include=['object']).columns
cardinalities = df[text_cols].nunique().sort_values(ascending=False)
top_card = cardinalities.head(5).index.tolist()

print("Variables con más necesidad de limpieza:")
print("Top 5 columnas con más faltantes:", top_missing)
print("Top 5 columnas de texto con más categorías únicas:", top_card)


Variables con más necesidad de limpieza:
Top 5 columnas con más faltantes: ['Unnamed: 1', 'TELEFONO', 'DIRECTOR', 'DIRECCION', 'SECTOR']
Top 5 columnas de texto con más categorías únicas: ['CODIGO', 'DIRECCION', 'TELEFONO', 'DIRECTOR', 'ESTABLECIMIENTO']


- Unnamed: 1 Columna vacía: debe eliminarse.
- ESTABLECIMIENTO Texto libre con mayúsculas/minúsculas inconsistentes, duplicados, errores ortográficos.
- DIRECCION	Espacios en blanco, caracteres especiales, faltantes, inconsistencias en formato.
- TELEFONO	Formatos variados guiones, paréntesis, espacios, valores faltantes, caracteres no numéricos.
- SUPERVISOR Nombres con mayúsculas/minúsculas mixtas, posibles nulos ocultos.
- DIRECTOR	Mismos problemas que SUPERVISOR, además de 25 faltantes.
- DISTRITO/MUNICIPIO Nombres con variantes ortográficas, acentos, mayúsculas.

# Limpieza de datos
## Estrategia de limpieza 
### obtener un DataFrame consistente eliminando columnas inútiles, homogeneizar texto y tratar valores faltantes de forma sencilla


In [23]:
# Eliminar columnas inutiles
cols_drop = ['Unnamed: 1', 'NIVEL', 'STATUS']
df1 = df.drop(columns=cols_drop)



In [24]:
# Normalizar nombres de columnas elimina eapcios en blanco y convierte a minúsculas
df1.columns = df1.columns.str.lower().str.strip()

In [25]:
# Direcciones y teléfonos faltantes
df1['direccion'] = df1['direccion'].fillna('SIN_DIRECCION')
df1['telefono']  = df1['telefono'].fillna('SIN_TELEFONO')
df1['director']  = df1['director'].fillna('SIN_DIRECTOR')

In [26]:
# Eliminar duplicados
df1 = df1.drop_duplicates()

In [27]:
# Unificar mayúsculas minúsculas y espacios
text_cols = ['distrito','departamento','municipio','establecimiento',
             'direccion','supervisor','director','sector','area',
             'modalidad','jornada','plan','departamental']
for c in text_cols:
    df1[c] = (df1[c]
              .astype(str)
              .str.strip()
              .str.lower()
              .str.replace(r'\s+', ' ', regex=True))

In [28]:
# Limpieza de teléfonos
import re

def clean_phone(x):
    x = str(x)  # Convertimos a string
    nums = re.sub(r'\D', '', x)
    return nums if len(nums) >= 8 else 'FORMATO_INV'

df1['telefono'] = df1['telefono'].apply(clean_phone)


En la variable telefono se realiza una limpieza que consiste en eliminar todos los caracteres no numéricos (como espacios, guiones, paréntesis o letras), dejando únicamente los dígitos. Si el resultado tiene menos de 8 dígitos, se considera un formato inválido y se reemplaza por 'FORMATO_INV'. Esto estandariza los números de teléfono y facilita su validación

In [29]:
# Conteo de formatos inválidos de teléfono
df1['telefono'].value_counts()['FORMATO_INV']



68

In [30]:
# ver lasfilas con formato inválido
df1[df1['telefono'] == 'FORMATO_INV'].head()

,codigo,distrito,departamento,municipio,establecimiento,direccion,telefono,supervisor,director,sector,area,modalidad,jornada,plan,departamental
83,16-03-0036-46,16-053,alta verapaz,san cristobal verapaz,instituto nacional de educacion diversificada,barrio chiyuc,FORMATO_INV,leonardo oxom chen,edy rolando jom coy,oficial,rural,monolingue,vespertina,diario(regular),alta verapaz
136,16-06-2398-46,16-011,alta verapaz,san miguel tucuru,instituto privado mixto de magisterio bilingue...,canton la playa,FORMATO_INV,ingrid amanda camó tobar de castro,miguel fernando tut lemus,privado,urbana,monolingue,vespertina,diario(regular),alta verapaz
151,16-07-0253-46,16-055,alta verapaz,panzos,colegio privado mixto juan amos comenio,colonia santa isabel,FORMATO_INV,maria elena palma molina,marcelino caz mucú,privado,rural,monolingue,doble,fin de semana,alta verapaz
170,16-09-0007-46,16-033,alta verapaz,san pedro carcha,centro educativo nueva vida,8a. calle 4-47 zona 4,FORMATO_INV,oscar rené ventura zac,juan rax,privado,rural,monolingue,vespertina,diario(regular),alta verapaz
207,16-09-9259-46,16-033,alta verapaz,san pedro carcha,"colegio ""sthella de hernandez""",diagonal 2 10-73 zona 2 barrio san pablo,FORMATO_INV,oscar rené ventura zac,orley edwin meléndez rosales,privado,urbana,monolingue,doble,fin de semana,alta verapaz


In [ ]:
# Eliminar tildes, comillas y caracteres especiales de establecimiento
df1['establecimiento'] = df1['establecimiento'].apply(lambda x: 
    re.sub(r'[^\w\s]', '',  
        unicodedata.normalize('NFD', x)  
        .encode('ascii', 'ignore')       
        .decode('utf-8')                 
    )
)


A la variable establecimiento se le aplicó una limpieza textual que incluyó: la conversión a minúsculas, eliminación de espacios duplicados y al inicio/final, eliminación de tildes y acentos mediante normalización Unicode, y la eliminación de signos de puntuación y caracteres especiales como comillas. Esto garantiza que los nombres de los establecimientos queden en un formato uniforme y comparable, reduciendo errores por variaciones tipográficas.